In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import torch
!pip install datasets
from datasets import load_dataset
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier

In [3]:
from transformers import AutoModel, DistilBertTokenizer

In [4]:
yelp = load_dataset('yelp_review_full')
# %%
yelp

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [5]:
train_ds=yelp['train'].select(range(1000))

In [6]:
model_name = 'distilbert-base-uncased'
device = 'cpu'


In [8]:
model=AutoModel.from_pretrained(model_name).to(device)
tokenizer=DistilBertTokenizer.from_pretrained(model_name)

In [10]:
text='hello i am super man or superman ?'
encoded_txt=tokenizer(text,return_tensors='pt')
encoded_txt

{'input_ids': tensor([[  101,  7592,  1045,  2572,  3565,  2158,  2030, 10646,  1029,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [11]:
tokens=tokenizer.convert_ids_to_tokens(encoded_txt['input_ids'][0])
tokens

['[CLS]', 'hello', 'i', 'am', 'super', 'man', 'or', 'superman', '?', '[SEP]']

In [15]:
tokenizer.convert_tokens_to_string(tokens)

'[CLS] hello i am super man or superman ? [SEP]'

In [16]:
tokenizer.vocab_size

30522

In [18]:
max_context_length = tokenizer.model_max_length
max_context_length

512

In [19]:
def tokenize_text(batch):
  return tokenizer(batch['text'],return_tensors='pt',padding='max_length',truncation=True)

In [21]:
yelp_encoding = train_ds.map(tokenize_text,batched=True,batch_size=128)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [23]:
yelp_encoding.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [32]:
yelp_encoding

Dataset({
    features: ['label', 'text', 'input_ids', 'attention_mask'],
    num_rows: 1000
})

In [24]:
tokenizer.model_input_names

['input_ids', 'attention_mask']

In [28]:
def get_last_hidden_state(batch):
    inputs = {k: v for k, v in batch.items() if k in tokenizer.model_input_names}
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state
        # [:, 0] refers to CLS token for complete sentence representation
    return {'hidden_state': last_hidden_state[:, 0]}

In [30]:
yelp_hidden_states = yelp_encoding.map(get_last_hidden_state, batched=True, batch_size=128)  # will have additional column 'hidden_state'

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [31]:
yelp_hidden_states

Dataset({
    features: ['label', 'text', 'input_ids', 'attention_mask', 'hidden_state'],
    num_rows: 1000
})

In [34]:
import joblib
joblib.dump(yelp_hidden_states, 'yelp_hidden_states.joblib')

['yelp_hidden_states.joblib']